In [5]:
import numpy as np
import pandas as pd
from math import log
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.linear_model import Ridge

import sys
sys.path.append('../utils')
from multiple_regression import *
from round_result import round_result


In [6]:
wine = pd.read_csv('../datasets/winequality-white.csv', sep=';')
wine['const'] = 1
wine['alcohol_log'] = wine['alcohol'].apply(lambda x: log(x))

X_train, X_test, y_train, y_test = train_test_split(wine.drop('quality', axis=1), wine['quality'], test_size=0.2, random_state=0)
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,const,alcohol_log
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,1,2.174752
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,1,2.251292
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,1,2.312535
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,2.292535
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,2.292535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1,2.415914
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1,2.261763
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1,2.240710
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1,2.549445


In [7]:
describe_multiple_regression_model(X_train, X_test, y_train, y_test, ['alcohol', 'fixed acidity', 'pH', 'residual sugar', 'free sulfur dioxide', 'sulphates'])


Avg train error =	0.5892111422964917
Avg validate error =	0.5873369417469086
Test error =		0.7163384268848929



In [8]:
hypothesis_features = ['const', 'alcohol', 'volatile acidity', 'density', 'alcohol_log']
considered_features = []

min_lambda = 0.003
lambda_multiplier_step = 2
max_lambda = 150

for feature in hypothesis_features:

    considered_features.append(feature)
    X_train_chosen = X_train[considered_features]

    current_lambda = min_lambda
    best_result = {'best_lambda': 0.00001, 'avg_train_error': 9999, 'avg_val_error': 9999, 'test_error': 9999}

    while current_lambda < max_lambda:

        ridge_model = Ridge(alpha=current_lambda, fit_intercept=False)
        X_train_scaled = standard_scaled(X_train_chosen, True)

        (avg_train_error, avg_val_error) = cross_validate(ridge_model, X_train_scaled, y_train)
        ridge_model.fit(X_train_scaled, y_train)
        test_error = round_result(y_test, ridge_model.predict(standard_scaled(X_test[considered_features], True)))[1]['actual_mean_sqrt']

        if test_error < best_result['test_error']:
            best_result['best_lambda'] = current_lambda
            best_result['avg_train_error'] = avg_train_error
            best_result['avg_val_error'] = avg_val_error
            best_result['test_error'] = test_error
                        
        current_lambda *= lambda_multiplier_step

    
    print(f"""Added: {feature} (lambda={best_result['best_lambda']})
    \tAvg train error =\t{best_result['avg_train_error']}
    \tAvg validate error =\t{best_result['avg_val_error']}
    \tTest error =\t\t{best_result['test_error']}
    """)

Added: const (lambda=49.152)
    	Avg train error =	0.7664878521630156
    	Avg validate error =	0.7699424034908015
    	Test error =		0.8813832757869122
    
Added: alcohol (lambda=49.152)
    	Avg train error =	0.6189826446907183
    	Avg validate error =	0.6244869050264605
    	Test error =		0.7265834331392841
    
Added: volatile acidity (lambda=49.152)
    	Avg train error =	0.5813728819130781
    	Avg validate error =	0.5825860825494514
    	Test error =		0.6826376071866341
    
Added: density (lambda=49.152)
    	Avg train error =	0.5766039120286803
    	Avg validate error =	0.575227268830109
    	Test error =		0.6782744784691089
    
Added: alcohol_log (lambda=49.152)
    	Avg train error =	0.577230141455302
    	Avg validate error =	0.5758812913259181
    	Test error =		0.6784588821515019
    
